In [23]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean
from statsmodels.tsa.arima.model import ARIMA

In [22]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 3.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

In [13]:
DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", "SELECT * FROM rent_price") \
    .load()

In [14]:
DF.show()

+----+---------+--------------+--------------------+--------------------+------+-------------------+
|Year|Trimester|      District|       Neighbourhood|       Average _rent| Price|      Price_Escalat|
+----+---------+--------------+--------------------+--------------------+------+-------------------+
|2014|        1|  Ciutat Vella|            el Raval|average rent (eur...|589.55|0.23641140585517476|
|2014|        1|  Ciutat Vella|      Gothic Quarter|average rent (eur...|712.79|0.30156053413404094|
|2014|        1|  Ciutat Vella|      la Barceloneta|average rent (eur...|540.71|0.21059281266189483|
|2014|        1|  Ciutat Vella|Sant Pere, Santa ...|average rent (eur...|673.44|0.28075869870907033|
|2014|        1|      Eixample|          Fort Pienc|average rent (eur...|736.09| 0.3138777581595001|
|2014|        1|      Eixample|     Sagrada Familia|average rent (eur...|673.37| 0.2807216941733715|
|2014|        1|      Eixample|la Dreta de l'Eix...|average rent (eur...| 921.4| 0.41183933

In [27]:
data = DF.toPandas()
data

,Year,Trimester,District,Neighbourhood,Average _rent,Price,Price_Escalat
0,2014,1,Ciutat Vella,el Raval,average rent (euro/month),589.55,0.236411
1,2014,1,Ciutat Vella,Gothic Quarter,average rent (euro/month),712.79,0.301561
2,2014,1,Ciutat Vella,la Barceloneta,average rent (euro/month),540.71,0.210593
3,2014,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",average rent (euro/month),673.44,0.280759
4,2014,1,Eixample,Fort Pienc,average rent (euro/month),736.09,0.313878
...,...,...,...,...,...,...,...
4617,2022,2,Sant Marti,Diagonal Mar i el Front Maritim del Poblenou,average rent per surface (euro/m2),17.50,0.790287
4618,2022,2,Sant Marti,el Besos i el Maresme,average rent per surface (euro/m2),12.20,0.497792
4619,2022,2,Sant Marti,Provencals del Poblenou,average rent per surface (euro/m2),14.60,0.630243
4620,2022,2,Sant Marti,Sant Marti de Provencals,average rent per surface (euro/m2),11.90,0.481236


In [31]:
data[data['Neighbourhood']=='el Raval'][data['Average _rent']==]

,Year,Trimester,District,Neighbourhood,Average _rent,Price,Price_Escalat
0,2014,1,Ciutat Vella,el Raval,average rent (euro/month),589.55,0.236411
68,2014,2,Ciutat Vella,el Raval,average rent (euro/month),550.63,0.215837
138,2014,3,Ciutat Vella,el Raval,average rent (euro/month),576.45,0.229486
206,2014,4,Ciutat Vella,el Raval,average rent (euro/month),597.00,0.240350
273,2014,1,Ciutat Vella,el Raval,average rent per surface (euro/m2),10.76,0.418322
...,...,...,...,...,...,...,...
4284,2021,4,Ciutat Vella,el Raval,average rent per surface (euro/m2),13.10,0.547461
4352,2022,1,Ciutat Vella,el Raval,average rent (euro/month),917.00,0.409513
4419,2022,1,Ciutat Vella,el Raval,average rent per surface (euro/m2),15.50,0.679912
4486,2022,2,Ciutat Vella,el Raval,average rent (euro/month),872.00,0.385725


In [20]:
data = data['Price_Escalat']
data

0       0.236411
1       0.301561
2       0.210593
3       0.280759
4       0.313878
          ...   
4617    0.790287
4618    0.497792
4619    0.630243
4620    0.481236
4621    0.453642
Name: Price_Escalat, Length: 4622, dtype: float64

In [24]:
data
train = data.iloc[:-12]  # Datos de entrenamiento (primeros 4 años)
test = data.iloc[-12:]   # Datos de prueba (último año)

# Paso 5: Entrenar modelo
model = ARIMA(train, order=(5,1,0))  # Especifica el orden del modelo ARIMA
model_fit = model.fit()

# Paso 6: Validar modelo
predictions = model_fit.forecast(steps=len(test))  # Hacer predicciones para los datos de prueba

# Calcula el error (por ejemplo, MSE)
mse = ((predictions - test) ** 2).mean()
print("Mean Squared Error:", mse)

# Paso 7: Hacer predicciones para el siguiente año
next_year_predictions = model_fit.forecast(steps=12)

Mean Squared Error: 0.03670318737621794


In [25]:
predictions

4610    0.504106
4611    0.500111
4612    0.492843
4613    0.494655
4614    0.497096
4615    0.497505
4616    0.498338
4617    0.497594
4618    0.497381
4619    0.497334
4620    0.497279
4621    0.497432
Name: predicted_mean, dtype: float64

In [26]:
test

4610    0.475717
4611    0.503311
4612    0.591611
4613    0.569536
4614    0.674393
4615    0.966887
4616    0.757174
4617    0.790287
4618    0.497792
4619    0.630243
4620    0.481236
4621    0.453642
Name: Price_Escalat, dtype: float64